In [14]:
import pandas as pd
import json as json

In [16]:
def loadFiles(version, arena):
    file_path     = "../data/"
    list_of_files = []
    arena_list    = ["arena-1", "arena-2", "arena-3", "arena-4", "arena-5", "arena-6", "arena-7", "arena-8", "arena-9",]
    file_path = getVersionPath(version, file_path)
    if arena in arena_list:   
        file_path        += "/" + arena + "/"
        encounter_pokemon = file_path + "EncounterablePokemon.csv"
        pokemon_of_leader = file_path + "LeaderPokemon.csv"
        arena_leader = file_path + "ArenaLeader.csv"
        move_of_in        = file_path + "MovesOfInterest.csv"
        list_of_files.append(encounter_pokemon)
        list_of_files.append(pokemon_of_leader)
        list_of_files.append(arena_leader)
        list_of_files.append(move_of_in)  
    else:
        print("[-] Arena not found...")
    return list_of_files

def getVersionPath(version, file_path):
    if version == "red":
        file_path += "red"
        return file_path
    elif version == "blue":
        file_path += "blue"
        return file_path

In [17]:
def loadStones(arena):
    #["moon-stone", "fire-stone", "thunder-stone", "water-stone", "leaf-stone"]
    ## TODO: add arena stone data
    list_of_stones = []
    
    arena1_stones = []
    arena2_stones =  arena1_stones + ["moon-stone"]
    arena3_stones = arena2_stones + []
    
    if arena == "arena-1":
        pass
    elif arena == "arena-2":
        for stone in arena2_stones:  
            list_of_stones.append(stone)
    elif arena == "arena-3":
        for stone in arena3_stones:
            list_of_stones.append(stone)
    return list_of_stones

In [274]:
# STATIC CSV FILES
pokemonTable = pd.read_csv('../Data/Pokemon.csv')
pokemonTypeTable = pd.read_csv('../Data/PokemonTypeChart.csv')

# SETUP ARENA-1
ARENA = 'arena-1'
GAME = 'blue'
csvPaths = loadFiles(GAME, ARENA)

encounterable_pokemonTable = pd.read_csv(csvPaths[0])

# [name, moves, encounter, originIfEvo, movesByVM, movesByTM]
encounterable_pokemonParsed = []

for row in encounterable_pokemonTable.values.tolist():
    row = row[1:]
    moves = row[1]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[1] = moves
    
    VMmoves = row[4]
    VMmoves = VMmoves.replace("'", "\"") 
    VMmoves = json.loads(VMmoves)
    row[4] = VMmoves
    
    TMmoves = row[5]
    TMmoves = TMmoves.replace("'", "\"") 
    TMmoves = json.loads(TMmoves)
    row[5] = TMmoves
   
    encounterable_pokemonParsed.append(row)
    
leader_pokemon =  pd.read_csv(csvPaths[1])

## um moves von leaderPokemon zu parsen
#test = leader_pokemon.loc[leader_pokemon['name']=='geodude']['moves'].values.tolist()[0].replace("'", "\"") 
#json.loads(test)

arena_leader = pd.read_csv(csvPaths[2])
arena_leader_parsed = []
for row in arena_leader.values.tolist(): 
    moves = row[3]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[3] = moves
    arena_leader_parsed.append(row[1:])

moves_of_interest = pd.read_csv(csvPaths[3])

## extracted Variables from DATAFRAME
AVERAGE_LEVEL = arena_leader['averageLevel'][0]

## Variables for each Arena
arena1_Options = loadStones(ARENA)

## get leaderPokemonData 
# [stats, moves]
leaderPokemonData = {}
for leaderPokemon in arena_leader_parsed[0][2]:
    moves = leader_pokemon.loc[leader_pokemon['name']==leaderPokemon]['moves'].values.tolist()[0].replace("'", "\"") 
    moves = json.loads(moves)
    moveDict = {}
    for move in moves:
        moveData = moves_of_interest.loc[moves_of_interest['name']==move]
        moveDict[move] = moveData
    leaderPokemonData[leaderPokemon] = [ extractPokemonStatsFromTable(leaderPokemon), moveDict] 


In [233]:
# extracts all data of interest for the enemy leader pokemon
def extractPokemonStatsFromTable(pokemonName):
    pokeRow = pokemonTable.loc[pokemonTable['Name'] == pokemonName]
    return pokeRow

In [192]:
# spuckt liste von Pokemon aus welche zur verf. stehen

# @para1: encounterable_pokemon - liste von antreffbaren pokemon :: [index,name,encounterOrEvolutionTrigger,...]
# @para2: averageLevel - averageLevel des ArenaLeiters
# @para3: stones_available - Die Entw. Steine welche zur verfuegung stehen
# @return: poss_team_member - liste von pokemon welche mit dem averageLevel und den vorhanden Steinen erreichbar sind :: [index,name,encounterOrEvolutionTrigger,...]
def possPokemonArenaNLeaderM(encounterable_pokemon, averageLevel, stones_available):
    
    encounterable_pokemon_available = []
    for pokemon in encounterable_pokemon:
        encounterOrEvolutionTrigger = pokemon[2]
        if encounterOrEvolutionTrigger.isnumeric() and int(encounterOrEvolutionTrigger) <= averageLevel:
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger == 'walk' or encounterOrEvolutionTrigger == 'starter' or encounterOrEvolutionTrigger == 'special' or encounterOrEvolutionTrigger == 'trade':
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger in stones_available:
            encounterable_pokemon_available.append(pokemon)
    return encounterable_pokemon_available

In [193]:
possPokemonArenaNLeaderM(encounterable_pokemonParsed, AVERAGE_LEVEL, arena1_Options)

[['pidgey',
  [{'name': 'gust', 'levelLearnedAt': 1},
   {'name': 'sand-attack', 'levelLearnedAt': 5},
   {'name': 'quick-attack', 'levelLearnedAt': 12},
   {'name': 'whirlwind', 'levelLearnedAt': 19},
   {'name': 'wing-attack', 'levelLearnedAt': 28},
   {'name': 'agility', 'levelLearnedAt': 36},
   {'name': 'mirror-move', 'levelLearnedAt': 44}],
  'walk',
  nan,
  [],
  []],
 ['rattata',
  [{'name': 'tackle', 'levelLearnedAt': 1},
   {'name': 'tail-whip', 'levelLearnedAt': 1},
   {'name': 'quick-attack', 'levelLearnedAt': 7},
   {'name': 'hyper-fang', 'levelLearnedAt': 14},
   {'name': 'focus-energy', 'levelLearnedAt': 23},
   {'name': 'super-fang', 'levelLearnedAt': 34}],
  'walk',
  nan,
  [],
  []],
 ['caterpie',
  [{'name': 'tackle', 'levelLearnedAt': 1},
   {'name': 'string-shot', 'levelLearnedAt': 1}],
  'walk',
  nan,
  [],
  []],
 ['metapod', [{'name': 'harden', 'levelLearnedAt': 1}], 'walk', nan, [], []],
 ['weedle',
  [{'name': 'poison-sting', 'levelLearnedAt': 1},
   {'name

In [164]:
def extractMoveData(moveName):
    return moves_of_interest.loc[moves_of_interest['name']==moveName]

In [204]:
# extracts available move names from pokemon list item
def calcAvailableMoveNamesForPokemon(ourPokemonAndMoves, averageLevel):
    moves = []
    # LEVEL UP MOVES
    for move in ourPokemonAndMoves[1]:
        if move['levelLearnedAt'] <= averageLevel:
            moves.append(move['name'])
    # VM MOVES
    for move in ourPokemonAndMoves[4]:
        moves.append(move)
    # TM MOVES
    for move in ourPokemonAndMoves[5]:
        moves.append(move)
    return moves

In [205]:
def createOurPokemonObject(pokemonName, pokemonMoveNames):
    moveData = []
    for move in pokemonMoveNames:
        moveData.append(extractMoveData(move))
    return [pokemonName, extractPokemonStatsFromTable(pokemonName),moveData]

In [286]:
def retreiveMoveEffectiveness(moveType, vsMoveType):
    return pokemonTypeTable.loc[pokemonTypeTable['type'] == moveType][vsMoveType].values[0]

In [294]:
avail_moves = calcAvailableMoveNamesForPokemon(encounterable_pokemonParsed[0], AVERAGE_LEVEL)

testPokeObject = createOurPokemonObject(encounterable_pokemonParsed[0][0], avail_moves)

In [350]:
def calcEffectiveMovesVs(ourPokeMoves, theirPoke):
    theirType1 = theirPoke[0]['Type 1'].values[0].lower()
    theirType2 = theirPoke[0]['Type 2'].values[0].lower()
    moveEffectivenessVsTheirType1 = {}
    for move in ourPokeMoves:
        moveType = move['moveType'].values[0]
        moveName =  move['name'].values[0]
        moveEffectivenessVsTheirType1[moveName] = retreiveMoveEffectiveness(moveType, theirType1)
    return(moveEffectivenessVsTheirType1)

In [349]:
calcEffectiveMovesVs(testPokeObject[2], leaderPokemonData['geodude'])

{'gust': 0.5, 'sand-attack': 2.0, 'quick-attack': 0.5}


In [ ]:
# Diese func. gibt eine list von Moves, von encounter_pokemon zurueck.
# @para1: poss_team_member - DataFrame von moeglichen Team-Membern
# @para2: name - name des Pokemon. Dieser wird benoetigt um seine moves zu suchen.
# @para3: lock - ist ein schloss fuer das anfuegen der Moves an einer liste 
# @return: move_list - gibt eine liste der moeglichen moves zurueck.

def getPokemonMoves(poke_team, name, lock):
    # list der moves eines Pokemon
    move_list = []

    poke = poke_team.loc[poke_team["name"] == name, ["moves"]]
    poke = poke.values[0][0]
    poke = poke.replace("'", "\"") 
    poke = json.loads(poke)
    
    # Die Tabellen EncounterablePokemon und LeaderPokemon speichern die Moves
    # unterschiedlich. Bei der EncounterablePokemon ist in der Spalte 'Moves' ein
    # weiters DataFrame enthalten, wo die moves der Pokemon unter dem Value 'name'
    # gespeichert sind. Deshalb muessen das auslesen dafuer etwas anders gemacht werden
    # Wenn lock == 0 ist werden die Moves unsers Teams der List hinzugefuegt 
    # Wenn lock == 1 ist werden die Moves des Arena-Teams der list hinzugefuegt
    if lock == 0:
        for move in poke:
            move_list.append(move["name"])

    elif lock == 1:

        for move in poke:
            move_list.append(move)
            
    return move_list